In [ ]:
import altair as alt

In [ ]:
from vega_datasets import data

cars = data.cars()

brush = alt.selection(type='interval', resolve='global')

base = alt.Chart(cars).mark_point().encode(
    y='Miles_per_Gallon',
    color=alt.condition(brush, 'Origin', alt.ColorValue('gray'))
).add_selection(
    brush
).properties(
    width=250,
    height=250
)

base.encode(x='Horsepower') | base.encode(x='Acceleration')

In [ ]:
import json as json
import nba_functions as nba
import numpy as np


In [ ]:
game1 = json.load(open('warriors_game1.json'))

In [ ]:
nba.positional_smoothing(game1)
nba.velo_accel_smoothing(game1)
nba.get_masses(game1)
nba.velo_accel(game1)
nba.smooth_energy(game1)
nba.ball_flagger(game1)
nba.player_flagger(game1)
nba.player_flagger2(game1)

In [ ]:
game = '2017/10/17/HOUGSW/'

x = [] #time (in minutes)
y = [] #total energy
c = []
names = []

for team in ['homeplayers','awayplayers']:
    
    for key in game1[game][team].keys():

        if team == 'homeplayers':
            c.append(200)
        else:
            c.append(0)

        en = np.array(game1[game][team][key]['work'])
        en_no_nan = en[en==en]
        energy = np.nansum(en_no_nan[en_no_nan<10000])

        oc = np.array(game1[game][team][key]['on_court'])

        
        timeplayed = len(oc[oc==False])/25/60
        y.append(energy/(10**6))
        x.append(timeplayed)
        names.append(str(key))
        
    

In [ ]:
from bokeh.io import show, reset_output
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.plotting import figure


reset_output()

album_names = names
valence = np.array(x)
energy = np.array(y)
radii =  np.ones(len(x))
colors = [
    "#%02x%02x%02x" % (int(g), int(g), int(g)) for r, g in zip(c,c)
]


source = ColumnDataSource(data=dict(valence=valence,
                                    energy=energy,
                                    names=album_names,
                                    radii = radii,
                                    colors = colors))

p = figure(tools=[HoverTool(tooltips=[('Player Name', '@names'),
                                      ('Minutes Played', '@valence'),
                                      ('Total Work (MJ)', '@energy')])])
p.scatter(x='valence', y='energy', radius='radii', fill_color = 'colors',source=source)


p.xaxis.axis_label = 'Minutes Played'
p.yaxis.axis_label = r'Energy (MJ)'

output_file("warriors_rockets_game1.html", title="Interactive")

show(p)